Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = False

In [5]:
# Read in sp600.csv
sp600 = pd.read_csv('sp600.csv')
sp600.drop(columns=['SEC filings', 'CIK'], inplace=True)
sp600.rename(columns={'Company':'Description',
                      'GICS Sector':'Asset Class',
                      'GICS Sub-Industry': 'GICS Sub Industry'}, inplace=True)
sp600.set_index("Symbol", inplace=True)
sp600

,Description,Asset Class,GICS Sub Industry,Headquarters Location
Symbol,,,,
AAP,"Advance Auto Parts, Inc.",Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina"
AAT,American Assets Trust,Real Estate,Diversified REITs,"San Diego, California"
ABCB,Ameris Bancorp,Financials,Regional Banks,"Atlanta, Georgia"
ABG,Asbury Automotive Group,Consumer Discretionary,Automotive Retail,"Duluth, Georgia"
ABM,"ABM Industries, Inc.",Industrials,Environmental & Facilities Services,"New York City, New York"
...,...,...,...,...
XRX,Xerox,Information Technology,"Technology Hardware, Storage & Peripherals","Norwalk, Connecticut"
YELP,"Yelp, Inc.",Communication Services,Interactive Media & Services,"San Francisco, California"
YOU,"Clear Secure, Inc.",Information Technology,Application Software,"New York City, New York"


In [6]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [7]:
# Map sp600 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp600['Asset Class'] = sp600.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
sp600.index = sp600.index.str.replace('.', '-', regex=False)
sp600

,Description,Asset Class,GICS Sub Industry,Headquarters Location
Symbol,,,,
AAP,"Advance Auto Parts, Inc.",US Stocks:Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina"
AAT,American Assets Trust,US Stocks:Real Estate,Diversified REITs,"San Diego, California"
ABCB,Ameris Bancorp,US Stocks:Financials,Regional Banks,"Atlanta, Georgia"
ABG,Asbury Automotive Group,US Stocks:Consumer Discretionary,Automotive Retail,"Duluth, Georgia"
ABM,"ABM Industries, Inc.",US Stocks:Industrials,Environmental & Facilities Services,"New York City, New York"
...,...,...,...,...
XRX,Xerox,US Stocks:Technology,"Technology Hardware, Storage & Peripherals","Norwalk, Connecticut"
YELP,"Yelp, Inc.",US Stocks:Communication Services,Interactive Media & Services,"San Francisco, California"
YOU,"Clear Secure, Inc.",US Stocks:Technology,Application Software,"New York City, New York"


In [8]:
# Drop invalid symbols.
sp600.drop(['DRQ', 'SGH', 'UCBI'], inplace=True)


In [9]:
# Make symbols list.
symbols = list(sp600.index)
#symbols

In [10]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

AAP AAT ABCB ABG ABM ABR ACA ACAD ACIW ACLS ADEA ADMA ADNT ADUS AEIS AEO AESI AGO AGYS AHCO AHH AIN AIR AKR AL ALEX ALG ALGT ALK ALKS ALRM AMBC AMN AMPH AMR AMSF AMTM AMWD ANDE ANIP AORT AOSL APAM APLE APOG ARCB ARI ARLO AROC ARR ARWR ASIX ASO ASTE ASTH ATEN ATGE ATI AUB AVA AVAV AVNS AWI AWR AX AXL AZTA AZZ BANC BANF BANR BBWI BCC BCPC BDN BFH BFS BGC BGS BHE BHLB BJRI BKE BKU BL BLFS BLMN BMI BOH BOOT BOX BRC BRKL BSIG BTU BXMT CABO CAKE CAL CALM 
Throttle limit reached. Waiting for 30 seconds...
CALX CARG CARS CASH CATY CBRL CBU CCOI CCS CENT CENTA CENX CERT CEVA CFFN CHCO CHEF CLB CNK CNMD CNR CNS CNXN COHU COLL CON COOP CORT CPF CPK CPRX CRC CRGY CRI CRK CRSR CRVL CSGS CSR CSWI CTKB CTRE CTS CUBI CURB CVBF CVCO CVI CWEN CWEN-A CWK CWT CXM CXW DAN DCOM DEA DEI DFH DFIN DGII DIOD DLX DNOW DOCN DORM DRH DV DVAX DXC DXPE DY EAT ECG ECPG EFC EGBN EIG ELME EMBC ENOV ENR ENVA EPAC EPC EPRT ESE ETD ETSY EVTC EXPI EXTR EYE EZPW FBK FBNC FBP FBRT FCF FCPT 
Throttle limit reached. Waiting fo

In [11]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df

,AAP,AAT,ABCB,ABG,ABM,ABR,ACA,ACAD,ACIW,ACLS,...,WT,WWW,XHR,XNCR,XPEL,XRX,YELP,YOU,ZD,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,142.24,30.68,29.21,68.12,27.82,5.54,27.59,16.65,27.14,17.79,...,5.95,28.31,14.35,35.79,NaN,14.49,34.57,NaN,59.27,22.46
2019-01-03,146.68,30.69,29.11,67.84,27.53,5.63,27.61,16.17,25.16,16.57,...,5.62,27.73,14.27,34.86,NaN,14.17,33.64,NaN,57.62,21.17
2019-01-04,143.05,30.94,30.25,68.47,28.47,5.78,28.32,17.57,26.17,17.61,...,5.77,28.32,14.79,36.58,NaN,14.72,34.56,NaN,60.02,22.01
2019-01-07,145.14,31.43,31.45,70.66,29.78,5.92,29.73,18.23,26.51,17.85,...,5.82,28.47,14.79,38.09,NaN,15.04,34.83,NaN,60.60,22.14
2019-01-08,144.14,32.07,31.37,69.83,31.11,6.02,29.50,18.54,27.39,17.78,...,5.98,28.89,15.29,38.01,NaN,15.53,35.51,NaN,60.88,22.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-27,49.22,24.92,65.10,271.12,53.09,13.61,97.16,18.27,53.25,69.45,...,9.85,23.22,15.05,19.23,42.52,9.71,40.35,24.05,54.51,38.89
2025-01-28,48.76,24.60,64.96,273.32,52.85,13.34,98.13,18.40,53.94,68.19,...,9.76,23.40,15.20,19.30,43.21,9.15,41.25,24.07,55.01,39.36
2025-01-29,48.74,23.77,65.21,274.31,53.00,13.38,99.05,18.37,53.69,68.61,...,9.85,23.12,14.83,18.83,42.24,8.62,41.04,23.37,53.95,38.96


In [12]:
# Sample symbol.
symbol = 'AAP'

In [13]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

np.float64(-26.462140853229954)

In [14]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [15]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

np.float64(0.33392592519976816)

In [16]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

np.float64(0.2036251147187177)

In [17]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()

In [18]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2019-01-31     NaN
2019-02-28    0.02
2019-03-31    0.05
2019-04-30   -0.02
2019-05-31   -0.07
              ... 
2024-09-30   -0.14
2024-10-31   -0.08
2024-11-30    0.16
2024-12-31    0.14
2025-01-31    0.03
Freq: ME, Name: AAP, Length: 73, dtype: float64

In [19]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

np.float64(0.4745582936098843)

In [20]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [21]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

# This is still slow (2.53 s).
for i, (index, row) in enumerate(sp600.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [22]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')